In [1]:
import os
import attk
import csv
import joblib
import time
import timeit

import os
import numpy as np
import librosa
from IPython.display import display, Audio
import timeit
import random
from sklearn.externals import joblib

from sklearn.ensemble import ExtraTreesClassifier

from IPython.display import display, Audio

random.seed(9998) ## Seeding random number generator for reproducibility

In [2]:
csv_path = "Potential_WGBH_tones_2887_classifier_4K_batch.csv"

completed_this_session=[]

In [3]:
######## RESUME HERE WHEN USING PRE-TRAINED MODEL ########

from sklearn.ensemble import ExtraTreesClassifier

extra_trees=joblib.load('aapb_test_tones_extra_trees.pkl')

##########################################################

In [4]:
## Creating a list of all media pathnames within a directory 
## (including subdirectories)

#media_paths = attk.find_media_paths('AAPB_Subset_4000/')

media_paths = ['AAPB_Subset_4000/'+item for item in os.listdir('AAPB_Subset_4000/') if item[-4:].lower() in ('.mp3','.mp4','.wav')]

media_paths[:3]+['...']

['AAPB_Subset_4000/cpb-aacip-86-20sqvd9q.h264.mp4',
 'AAPB_Subset_4000/cpb-aacip-77-02q57zt0.h264.mp4',
 'AAPB_Subset_4000/cpb-aacip-15-rr1pg1jc1z__IHUB-A_.mp3',
 '...']

In [5]:
## Creating a list of files that have already been classified (in case we get interrupted)

completed_filenames=[]

with open(csv_path) as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        try:
            completed_filenames.append(row[0])
        except:
            print("Problem loading: "+str(row))



In [6]:
try: os.mkdir('temp_clip')
except: pass

In [7]:
from copy import copy

def update_completed_list():
    completed_filenames = copy(completed_this_session)

    with open(csv_path) as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
            try:
                completed_filenames.append(row[0])
            except:
                print("Problem loading: "+str(row))

update_completed_list()
print len(list(set(completed_filenames)))

4834


In [ ]:
## Function version of the script above, for multithreading

def classify_and_save(media_path):
    
    ## Starting timer
    tic=timeit.default_timer()
    
    completed_this_session.append(media_path.split('/')[-1])
    
    try:
        if media_path.split('/')[-1] not in completed_filenames: ## Checking whether file
                                                                 ## been classified already

            ## Creating temporary WAV excerpt and extracting features
            clip_pathname = attk.subclip(media_path,0,80,'temp_clip/')  ## first 80 seconds
            test_mfccs = attk.get_mfccs_and_deltas(clip_pathname)
            
            ## Classifying audio
            results = extra_trees.predict(test_mfccs)
            print(list(results[:150])+['...'])
            
            ## Smoothing and filtering output
            smoothed_ranges = attk.labels_to_ranges( \
                        [round(item) for item in attk.smooth(results, window_len=10)], \
                        label=0)
            
            confirmed_tones=[]
            
            if len(smoothed_ranges)>0:
                for pair in smoothed_ranges:
                    start, end = pair
                    if (end-start) > 5:
                        start_time = (80.0/len(results))*start         
                        end_time = (80.0/len(results))*(end+5)
                        confirmed_tones.append((start_time,end_time))
            
            ## Writing positive classification ranges to CSV
            with open(csv_path,'a') as csvfile:
                csvwriter = csv.writer(csvfile)
            
                for pair in confirmed_tones:
                    start, end = pair
                    csvwriter.writerow([media_path.split('/')[-1],str(round(start,4)),str(round(end,4))])
            
                if len(confirmed_tones)==0:
                    csvwriter.writerow([media_path.split('/')[-1],'',''])
            
            ## Deleting temporary WAV clip
            try: os.remove(clip_pathname)
            except: print "** Error: Can't remove "+clip_pathname

    ## Printing and storing errors
    except Exception as e:
        error_pair=(media_path,e)
        print(error_pair)
        errors.append(error_pair)
        try: os.remove(clip_pathname)
        except: pass
    
    time.sleep(random.random()*3)
    
    ## Printing time elapsed
    print("Time elapsed: "+str(timeit.default_timer() - tic))
    if float(timeit.default_timer() - tic) > 0.5:
        print "***"



In [ ]:
%%capture

# Classifying in parallel batches of 4

errors=[]

import time
from multiprocessing import Pool

pool_size = 4

pool = Pool(pool_size)

for i in range(1000):
    tic=timeit.default_timer()
    pool_media_paths=[]
    
    #pool = Pool(pool_size)
    
    for j in range(pool_size):
        media_path = random.choice(media_paths)
        update_completed_list()
        counter=0
        while media_path.split('/')[-1] in completed_filenames:
            media_path = random.choice(media_paths)
            counter+=1
            if counter>10000:
                print("Looks like you might be done.")
                break
                break
                break
        pool_media_paths.append(media_path)

    print(pool_media_paths)
    pool.map(classify_and_save, pool_media_paths)
    print("Time elapsed: "+str(timeit.default_timer() - tic))


In [ ]:
print("Time elapsed: "+str(timeit.default_timer() - tic))

In [ ]:
print(errors)